In [ ]:
import numpy as np
import pandas as pd
import dataset
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
from model import JetTransformer
from helpers_train import *
import matplotlib.pyplot as plt
import ipykernel

device = "cpu"

In [ ]:
testModel = JetTransformer()

testModel = load_model_checkpoint("output/checkpoints/TTBar_600000_best.pt")

Added start token!
Added stop token!
Max bins without padding: Num_bins: [42, 32, 32]
---> Number of different pt, eta, phi without padding: Total_voc_bin: [44, 34, 34]
Bins reserved for PAD: [43, 33, 33]


Args used for training: {'data_path': 'processed_data/TTBar_5000_train.h5', 'num_const': 20, 'add_start': True, 'add_stop': True, 'num_epochs': 20, 'lr': 0.001, 'hidden_dim': 256, 'num_layers': 8, 'num_heads': 4, 'dropout': 0.1, 'n_pt': 40, 'n_eta': 30, 'n_phi': 30, 'causal_mask': True, 'output_path': 'output/', 'name': 'debug', 'contin': False, 'batch_size': 20, 'n_jets': 100, 'n_jets_val': 60}
Added start token!
Added stop token!
Max bins without padding: Num_bins: [42, 32, 32]
---> Number of different pt, eta, phi without padding: Total_voc_bin: [44, 34, 34]
Bins reserved for PAD: [43, 33, 33]


In [3]:
testSet = DataLoader(
    JetDataSet("processed_data/TTBar_5000_test.h5", "test"),
    batch_size=100
)

Loading Dataset: test | n_jets = None | processed_data/TTBar_5000_test.h5
Added start token. New bins are now: [41, 31, 31]
Added stop token. New bins are now: [42, 32, 32]


In [4]:
probabilities = np.array([])

for x in testSet:
    
    logits = testModel.forward(x)
    probs = testModel.probability(logits, x, logarithmic= True, topk = 5000)

    probabilities = np.append(probabilities, probs.detach().numpy())

AttributeError: module 'model' has no attribute 'forward'

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4), dpi = 150)

ax.hist(
    probabilities,
    bins=100,                 # adjust for smoothness
    density=True,            # normalization
    histtype="step",
    linewidth=1.0,
    color="black",
    label="Top"
)

# axis labels
ax.set_xlabel(r"$\log(p)$")
ax.set_ylabel("Normalized distribution")
ax.set_title("Top")

# ticks and frame styling
#ax.tick_params(axis="both", which="major", labelsize=14, width=1.8, length=7)
#for spine in ax.spines.values():
#    spine.set_linewidth(1.8)

# legend in upper-left with box
ax.legend(
    #loc="upper left",
    frameon=True,
    framealpha=1.0,
    #fontsize=18,
    borderpad=0.8,
    handlelength=2.5
)

# optional: match x-range style from example
#ax.set_xlim(probabilities.min(), 0)

#plt.tight_layout()
plt.show()